In [97]:
# Neural network to identify the flowers type using the Iris data 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from sklearn import linear_model, metrics, model_selection, datasets

def sigmoid(z):
    """The sigmoid function.
    """
    return 1.0 / ( 1.0 + np.exp(-z))

def devsigmoid(z):
    """Derivative of the sigmoid function.
    """
    return sigmoid(z) * (1 - sigmoid(z))

class arrays(object):
    """To generate the ramdon matrices for each layer
    """
    def __init__(self, row=None, col=None,seeds=None):
        self.row = row
        self.col = col
        self.seeds= seeds
          
    def matrix(self):
        np.random.seed(self.seeds)
        if self.col == 1:
            return np.random.rand(self.row)
        elif self.row == 1:
            return np.random.rand(self.col)
        else:
            return np.random.rand(self.row, self.col)
        
    def vec(self):
        np.random.seed(77 * self.seeds + 29)
        return np.random.rand(self.row)
    
def resta(v1, v2):
    """Function to substract vectors of any size.
        len(v1) = len(v2)
    """    
    if len(v1) > 1:
        vec=[]
        for i in range(len(v1)):
            vec.append((v1[i] - v2[i]))
    else:
        vec = np.array(v1) - np.array(v2)
    return np.array(vec)

def transres(values, options):
    """Transform data input results into vectors of
       the size of the element of the last layer
    """    
    if options == 1 :
        return np.array(values)
    else:
        newvec = np.zeros(options)
        newvec[values] = 1
        return newvec

#**************************************************************#
#---------------------- back_propagationagation -----------------------#
#------------------------------------------------ -------------#   
def back_propagation(M, bb, valout, y, yp, siz, eta):
    """ Input:
            M = coeficcients matrix
            bb = bias 
            valout = value predicted
            y = data input (previous value got from feed_forwardard)
            yp = value expected 
        Return    
            M = list of ndarrays. It contains the calculated weigths for each layer (for one epoch)
            bb = list of ndarrays. It contains the calculated bias for each layer (for one epoch)

    """
#--------------------------- Beginning ------------------------#
#------------------- Code to get the first delta --------------#
    dim0 = np.reshape(M[siz - 2], (len(M[siz - 2]), -1))
    xsiz, ysiz = np.shape(dim0)
    cost = resta(valout, yp)
    if ysiz <= 1:
        z = np.dot(M[siz - 2].T, y[siz - 2]) + bb[siz - 2]
        delta = 2 * cost * devsigmoid(z)
    else:  
        z = np.matmul(M[siz - 2], y[siz - 2]) + bb[siz-2]
        z1 = len(z)
        delta0 = []
        for i in range(z1):
            delta0.append(2 * cost[i] * devsigmoid(z[i]))
        delta = np.reshape(np.array(delta0), (z1, 1)) 
#------------------------------------------------------------------#
    for k in range(siz - 2, -1, -1):
        nabla_w1 = []
        y2 = y[k]
        siz1, siz2 = len(y2), len(delta)
        bb[k] = np.array(bb[k])
        for i in range(siz1):
            for j in range(siz2):
                nabla_w1.append(eta * y2[i] * delta[j]) 
        nabla_w = np.reshape(np.array(nabla_w1), (siz1, siz2))        
        nabla_b = (eta * delta)
        if len(np.atleast_1d(delta)) == 1:
            suma = M[k] * delta
            M[k] = (M[k] - nabla_w )
        else:
            suma = np.matmul(M[k].T, delta)
            M[k] = (M[k] - nabla_w.T)
        bb[k] = bb[k] - nabla_b
        if k >= 1 :
            y2 = np.reshape(y[k - 1], (len(y[k - 1]), 1))
            z = np.matmul(M[k - 1], y2) + bb[k - 1]
            junto = []
            for j in range(len(z)):
                junto.append(suma[j] * devsigmoid(z[j]))
            delta = np.reshape(np.array(junto), (len(z), 1))
            nabla_w = []       
    return (M, bb)   

#**************************************************************#
#----------------------- feed_forwardard --------------------------#
#--------------------------------------------------------------# 
def feed_forward(w, vals, bbb, size_sys):
    """ 
        Input:
            w = list of ndarrays. It contains the calculated weigths for each layer
            vals = numpy array. Row from the dataset 
            bbb = list of ndarrays. It contains the calculated bias for each layer
            size_sys = integer. Number of hidden layers 
        Return:
            fval = numpy array. It contains the output of each layer
    """  
    fval = []
    vals = np.reshape(vals, (len(vals), 1))
    for j in range(size_sys - 1):
        w[j] = np.array(w[j])
        if len(np.atleast_1d(bbb[j])) == 1:
            valor = []
            bbb[j] = bbb[j]
            w[j] = np.reshape(w[j], (len(w[j]), -1))
            vals = np.dot(w[j].T, vals) + np.array(bbb[j])
            for i in range(len(vals)):
                valor.append(sigmoid(vals[i]))
            vals = np.array(valor) 
        else:
            valor = []
            bbb[j] = np.reshape(bbb[j], (len(bbb[j]), 1))
            vals = np.matmul(w[j], vals) + np.array(bbb[j])
            for i in range(len(vals)):
                valor.append(sigmoid(vals[i]))
            vals = np.array(valor)  
        fval.append(vals)
    fval = np.array(fval)   
    return fval

#------------------------------------------------------------------------#
#***************************  Neural Network   **************************# 
#------------------------------------------------------------------------#
#------------------------------------------------------------------------#
def network(vec, indat, res, eta, tol, epochs=1000):
    """ vec is a vector that contains the number of neurons in each layer. 
        The input parameters are considered as the first layer 
        Input:
            vec = list containing the neural network geometry
            indat = numpy array. It contains the training dataset
            res = numpy array. It contains the target of the dataset
            eta = float. step in the gradient descent
            tol = float. Error tolerance
            epochs = integer. Number of epochs, set by default to 1000
        Return:
            wf = list of ndarrays. It contains the calculated weigths for each layer
            wb = list of ndarrays. It contains the calculated bias for each layer
    """
    siz = len(vec)
    sizdata = len(indat)
    vals= np.array(indat) 
    wi, bi = [], [] 
    
#--------------------- initialization ---------------------#  
    for j in range(siz - 1):
        mm = arrays(vec[j + 1], vec[j], j + 7)
        ww, bb = mm.matrix(), mm.vec()
        wi.append(ww)
        bi.append(bb)  
#-----------------------------------------------------------       
    epoca = 0 
    while True:
        eerr = True
        for kk in range(sizdata):
            yi = [vals[kk,:]]          
            vals1 = vals[kk]
            results = transres(res[kk], vec[siz - 1])
            y2 = feed_forward(wi, vals1, bi, siz)
            for i in range(len(y2)):
                yi.append(y2[i])
            respred = yi[siz - 1]
            erro = resta(respred, results)
            erreur = (erro**2).mean()
            if erreur >= tol:
                wi, bi = back_propagation(wi, bi, respred, yi, results, siz, eta)
                eerr = True
            wf, bf = wi, bi       
        epoca += 1 
        if (epoca % 50 == 0 ):
            print('epoca: ', epoca, ', error: ', erreur)
            
        if eerr == False or epoca == epochs:
            break                     
    return (wf, bf)

In [98]:
iris = datasets.load_iris()
x_train, x_test, y_train, y_test = model_selection.train_test_split(iris.data,iris.target, test_size=0.2, random_state=0)

Nlayer1 = len(x_train[0]) # To know the size of the first layer
red = [Nlayer1, 3, 3, 3] # Geometry of the Neural network (last value should be the number of classes in the dataset)
eta, tol = 0.05, 1e-4
print("The results start here")
wajust, bajust = network(red, x_train, y_train, eta, tol, 1000)


def predict(x_teste, wajust, bajust):
    """ prediction. It output the probabilities of belonging to a determined class
        Input:
            x_teste = numpy array. Target of the test set.
            wajust = list of ndarrays. Weigth matrix for each layer
            bajust = list of ndarrays. bias array for each layer
        Return: 
            sol = list of numpy arrays. It containts the probability for each class for each line in x_teste
    """
    siz = np.shape(x_teste)
    layersnum = len(red)
    vals = np.array(x_teste)
    sol = []
    for k in range(siz[0]):
        vals1 = vals[k] 
        values = feed_forward(wajust, vals1, bajust, layersnum)
        sol.append(values[layersnum-2])
    sol = np.array(sol)    
    return sol

iden = predict(x_test, wajust, bajust)

def accuracy(y_pred, y_true):
    """ Function to calculate the accuracy of the model
    """
    total = 0
    size = len(y_true)
    yf = []
    for i in range(size):
        lista = (y_pred[i].T).tolist()[0]
        value = lista.index(max(lista))
        yf.append(value)
        if value ==  y_true[i]:
            total += 1
    return round(total*100 / size, 2), yf 

error, yf = accuracy(iden, y_test)
print(yf,'predicted values')
print(y_test, 'True values')
print('accuracy of the method: ', str(error) + ' %') 

The results start here
epoca:  50 , error:  0.21874157400801075
epoca:  100 , error:  0.2191581322379285
epoca:  150 , error:  0.21964107496965224
epoca:  200 , error:  0.21998556012335255
epoca:  250 , error:  0.12062354571939149
epoca:  300 , error:  0.011341022504714643
epoca:  350 , error:  0.024747153049116354
epoca:  400 , error:  0.013012578418722798
epoca:  450 , error:  0.007680585283179974
epoca:  500 , error:  0.006351039463018221
epoca:  550 , error:  0.004221222087347685
epoca:  600 , error:  0.0034504111109376102
epoca:  650 , error:  0.003721056752346078
epoca:  700 , error:  0.0024046452822704214
epoca:  750 , error:  0.0013718438112620462
epoca:  800 , error:  0.0037840815822902034
epoca:  850 , error:  0.002106895636745465
epoca:  900 , error:  0.002736074800261605
epoca:  950 , error:  0.0029510224172958834
epoca:  1000 , error:  0.0028659471189049044
[2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1, 0, 0, 2, 0, 0, 1, 1, 0] predicted values
[2 1 0 2 